In [679]:
import requests
import urllib
import numpy as np
import math
from bs4 import BeautifulSoup
from collections import Counter
import nltk
import string
#nltk.download()
stop_words = set(nltk.corpus.stopwords.words('english'))

In [2]:
blocklist = [
    "Main_Page",
    "Help:",
    "Special:",
    "Portal:",
    "Talk:",
    "Template:"
]

In [691]:
def nltk_pipeline(words, ngram_size=1):
    tokens = nltk.tokenize.word_tokenize(words)
    tokens = [w.lower() for w in tokens]
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    words = [word for word in stripped if word.isalpha()]
    words = [w for w in words if not w in stop_words]
    words = nltk.ngrams(words, ngram_size)
    return Counter(words)

In [686]:
def parse_wiki_link(link, get_text=False):
    r = requests.get(link)
    
    soup = BeautifulSoup(r.content, 'html.parser')
    # jank way to figure out redirect links
    real_link = soup.find_all('link', {"rel" : "canonical"})[0].get("href").split("#")[0]
    if real_link == link:
        real_link = None

    wiki_content_links = set()
    for link in soup.find_all('a', href=True):
        if link["href"] == "#cite_ref-1":
            break
        clean_link = link["href"]
        clean_link = urllib.parse.unquote(clean_link)
        if clean_link.startswith("/wiki/"):
            wiki_link = clean_link[6:]
            if any(x in wiki_link for x in blocklist):
                continue
            wiki_content_links.add(clean_link)

    if get_text:
        # todo - clean this text
        words = soup.find_all('p')
        return wiki_content_links, real_link, words

    return wiki_content_links, real_link

In [708]:
def parse_wiki_api(link, get_text=False):
    S = requests.Session()

    URL = "https://en.wikipedia.org/w/api.php"

    PARAMS = {
        "action": "parse",
        "format": "json",
        "prop": "links|properties",
        "redirects": ""
    }
    PARAMS["page"] = link
    if get_text:
        PARAMS["prop"] += "|wikitext"

    R = S.get(url=URL, params=PARAMS)
    DATA = R.json()

    if "parse" not in DATA.keys():
        return None
    parse_links = DATA["parse"]["links"]
    pageid = DATA["parse"]["pageid"]
    redirect_list = DATA["parse"]["redirects"]
    if len(redirect_list) > 0:
        redirect = redirect_list[0]['to']
    else:
        redirect = None
    links = []
    for l in parse_links:
        links.append(l['*'])

    if get_text:
        words = DATA["parse"]["wikitext"]["*"]
        words = nltk_pipeline(words)
        return links, pageid, redirect, words
    return links, pageid, redirect

In [709]:
class UserHistory:
    def __init__(self, user_history):
        self.user_history = user_history
        self.already_visited_pages = set() # resolves redirects in user_history

        # user_vists is a list of links in chronological order ascending
        # user_vists[-1] is the current page
        self.outgoing_links = Counter()
        #self.ingoing_links = set()
        
        self.words = Counter()
        for link in set(user_history):
            out_links, pageid, redirect, words = parse_wiki_api(link, get_text=True)
            if redirect is not None:
                self.already_visited_pages.add(redirect)
            else:
                self.already_visited_pages.add(link)

            self.words.update(words)
            self.outgoing_links.update(set(out_links))
            #self.ingoing_links.update(parse_wiki_ingoing(link))

        # remove self-loops
        for page in self.already_visited_pages:
            if page in self.outgoing_links:
                del self.outgoing_links[page]
        #self.ingoing_links -= already_visited_pages
        
        outgoing_links_list = list(self.outgoing_links.values())
        self.mean = np.average(outgoing_links_list)
        self.std = np.std(outgoing_links_list)

In [694]:
text = parse_wiki_api("Albert Einstein", get_text=True)[-1]

In [609]:
def wiki_prefix(suffix):
    # suffix is '/wiki/<article title>'
    return "https://en.wikipedia.org/"+suffix

def wiki_title_from_link(wiki_link):
    # wiki_link format: "https://en.wikipedia.org/wiki/Hamilton–Jacobi–Bellman_equation"
    title_unform = wiki_link[30:]
    title = title_unform.replace("_"," ")
    return title

In [610]:
class Cache:
    def __init__(self, fetch_fn):
        self.dict = dict()
        self.fetch_fn = fetch_fn

    def __call__(self, key, args):
        if key in self.dict:
            return self.dict[key]
        result = self.fetch_fn(args)
        self.dict[key] = result
        return result

def linkcount_fetch(wiki_page):
    # this is so damn slow
    link = f"https://linkcount.toolforge.org/api/?page={wiki_page}&project=en.wikipedia.org"
    r = requests.get(link).json()
    return r["wikilinks"]["all"]

In [611]:
def score_link_similarity(user_history, target):
    # user_history  
    #   incorporate idf (just hyperlinks) -> scrape target/what_links_here (expensive)
    #   or sample 10000 pages and count link frequency and store it somewhere else                       
    #   incorporate ingoing recommendations
    # return score(target | user_history)

    # how many times does target appear in self.outgoing_links
    z_score = (user_history.outgoing_links[target]-user_history.mean)/user_history.std
    return .5 * (math.erf(z_score / 2 ** .5) + 1)

In [732]:
def score_link_text_similarity(user_history, target):
    # user_history  
    #   incorporate idf (text)
    #   incorporate ingoing recommendations
    # return score(target | user_history)

    # references in div class = reflist
    target = nltk_pipeline(target)
    words = user_history.words
    total = 0
    for term in target:
        total += words[term]
    return total

In [860]:
def score_coupling_similarity(user_history, target, cache, doc_freq_cache):
    # user_history  
    #   need to download target and scrape it's links
    # pages are similar if their outgoing (ingoing) links have overlap
    if target in cache:
        results = cache[target]
    else:
        results = parse_wiki_api(target, get_text=False)
        # TODO swap to MediaWiki API
        cache[target] = results
    if results is None:
        # likely that this link doesn't exist
        return None
    links, pageid, redirect = results
    if redirect is not None and redirect in user_history.already_visited_pages:
        # don't recommend this page, a redirected variant was in the user history
        # (only way to resolve redirects from outgoing-links is through this api call)
        return None

    # TODO: implement faster doc freq
    #doc_freq = doc_freq_cache()
    target_outgoing = Counter(links)

    score = 0
    doc_len = sum(v for v in user_history.outgoing_links.values())

    # BM25 hyperparameters that are untuned
    k1 = 0.5
    k3 = 0.99
    b = 0.9
    avg_doc_len = 50 # estimate?
    for link, count in target_outgoing.items():
        query_count = user_history.outgoing_links[link]
        if count == 0 or query_count == 0:
            continue
        #page_name = link.split("/wiki/")[1]
        doc_freq = 100 #doc_freq_cache(page_name, page_name)

        norm_qtf = (k3+1)*query_count / (k3 + query_count)
        norm_tf = count * (k1 + 1) / (count + k1*((1-b)+b*(doc_len/avg_doc_len)))
        tf = norm_tf * norm_qtf

        num_links_on_wiki = 1
        idf = 1 #np.log(num_links_on_wiki / (doc_freq+1))
        score += tf * idf
    #union = sum(v for v in target_outgoing.values()) + sum(v for v in user_history.outgoing_links.values())

    return score

In [861]:
def compute_outgoing_scores_baseline(user_history):
    # composite score_link_similarity and score_link_text_similarity
    # (todo: this filters scores, will do re-ranking with coupling similarity, re-ranking with deeper searches, etc)
    weight = 0.5 # to be tuned
    outgoing_scores = dict()
    for link in user_history.outgoing_links:
        link_sim = score_link_similarity(user_history, link)
        text_sim = score_link_text_similarity(user_history, link)
        outgoing_scores[link] = link_sim + weight * text_sim
    return outgoing_scores

In [875]:
def rerank_with_coupling(user_history, baseline_scores, num_rerank):
    new_rankings = {k:v for k, v in baseline_scores}
    for target, score in baseline_scores[:num_rerank]:
        new_score = score_coupling_similarity(user_history, target, cache, doc_freq_cache)
        if new_score is None:
            continue
        new_rankings[target] = new_score*5 + score
    return new_rankings

In [876]:
def recommend_from_history(user_history):
    baseline_scores = compute_outgoing_scores_baseline(user_history)
    sorted_baseline_scores = [(k, v) for k, v in sorted(baseline_scores.items(), reverse=True, key=lambda item: item[1])]
    final_results = rerank_with_coupling(user_history, sorted_baseline_scores, 8)
    sorted_final_results = [k for k, v in sorted(final_results.items(), reverse=True, key=lambda item: item[1])]
    return sorted_final_results

In [ ]:
def visualize_history(user_history):
    pass

In [877]:
cache = dict()
doc_freq_cache = Cache(linkcount_fetch)

In [878]:
link1 = wiki_title_from_link("https://en.wikipedia.org/wiki/Hamilton–Jacobi–Bellman_equation")
link2 = wiki_title_from_link("https://en.wikipedia.org/wiki/Value_function")
link3 = wiki_title_from_link("https://en.wikipedia.org/wiki/Optimal_control")
user_history = UserHistory([link1, link2, link3])

In [871]:
link1 = wiki_title_from_link("https://en.wikipedia.org/wiki/Bitcoin")
link2 = wiki_title_from_link("https://en.wikipedia.org/wiki/Lightning_Network")
user_history = UserHistory([link1, link2])

In [879]:
recommend_from_history(user_history)

['Pseudospectral optimal control',
 'Control theory',
 'DIDO (optimal control)',
 'Control (optimal control theory)',
 'Hamiltonian (control theory)',
 'Optimal control theory',
 'Talk:Optimal control',
 'Stochastic control',
 'Control strategy',
 'Model Predictive Control',
 'Sliding mode control',
 'Linear-quadratic-Gaussian control',
 'Partial differential equation',
 'Elliptic partial differential equation',
 'Optimization problem',
 'Template:Cite journal',
 'Bellman equation',
 'Differential equation',
 'Indirect utility function',
 'Riccati equation',
 'Objective function',
 'Function (mathematics)',
 'Costate equation',
 'Boundary-value problem',
 'Differentiable function',
 'Difference equation',
 'Measurable function',
 'Lyapunov function',
 'Dynamic programming',
 'Brachistochrone problem',
 "Merton's portfolio problem",
 'Loss function',
 'Hamilton–Jacobi equation',
 'Variational problem',
 'Value (mathematics)',
 'Bellman pseudospectral method',
 'Discrete time and continu